In [ ]:
#Step 0. User Input - Datset of Interest
hypothesis = 2

if hypothesis == 1:
    fileName = 'williamsBrainstormChallenge_T1vsT2Features.csv'
elif hypothesis == 2:
    fileName = 'williamsBrainstormChallenge_T1vsT3Features.csv'
else:
    print('Analysis not supported. Please use hypothesis = 1 or hypothesis = 2.')
    
#Step 1. Load Modules
import numpy as np
import matplotlib.pyplot as plt
from autora.skl.darts import DARTSRegressor, ValueType, DARTSExecutionMonitor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
import copy
from IPython.display import clear_output
import pickle
import time

# Step 2. Load Data, Split Into Predictors and Outcomes, and Normalize Predictors
#Load Data 
featuresData = np.genfromtxt(fileName, delimiter=',')
scaler = StandardScaler()
featuresData[:,1:] = scaler.fit_transform(featuresData[:,1:])

#Split data into predictors and outcomes
outcomes = featuresData[:,0]
predictors = featuresData[:,1:]

# Step 3. Iterate Through Data Using the Leave-One-Out Metho
PRIMITIVES = [
    "none",
    "linear",
    "linear_logistic",
    "linear_relu",
    "linear_sin",
    "linear_tanh"
    ]

#The main loop searching for a DARTS architecture
avgAccuracies =[]
x=1
for run in range(1000):
    accuracies = []
    print((x/1000)*100)
    x+=1

    # General DARTS meta-parameters
    num_graph_nodes = 1
    max_epochs = 300
    sampling_strategy = 'max'

    # Meta-parameters for the architecture updates
    arch_updates_per_epoch = 1
    arch_learning_rate_max = 0.01
    arch_weight_decay = 3e-4
    arch_weight_decay_df = 0.01

    # Meta-parameters for the parameter updates
    param_updates_per_epoch = 100
    param_learning_rate_max = 0.025
    param_learning_rate_min = 0.01
    param_weight_decay = 3e-4
    param_momentum = 0.90
    param_updates_for_sampled_model = 1000

    # Meta-parameters for the classifier
    train_classifier_bias=True,
    train_classifier_coefficients=True

    # we will add this monitor to display the training performance
    execution_monitor_0 = DARTSExecutionMonitor()

    # run DARTS
    darts_estimator = DARTSRegressor(
        execution_monitor = execution_monitor_0.execution_monitor,
        num_graph_nodes = num_graph_nodes,
        max_epochs = max_epochs,
        arch_updates_per_epoch = arch_updates_per_epoch,
        arch_learning_rate_max = arch_learning_rate_max,
        arch_weight_decay = arch_weight_decay,
        arch_weight_decay_df = arch_weight_decay_df,
        param_updates_per_epoch = param_updates_per_epoch,
        param_learning_rate_max = param_learning_rate_max,
        param_learning_rate_min = param_learning_rate_min,
        param_weight_decay = param_weight_decay,
        param_momentum = param_momentum,
        train_classifier_bias = train_classifier_bias,
        train_classifier_coefficients = train_classifier_coefficients,
        output_type=ValueType.PROBABILITY,
        primitives=PRIMITIVES,
    )

    #Fit DARTS
    darts_estimator.fit(predictors, outcomes)

    #Saving architecture
    fixedArchitecture = copy.deepcopy(darts_estimator.network_)
    fullArchitecture = copy.deepcopy(darts_estimator)

    #Leave-one-out cross-validation methods to determine architecture accuracy
    for participant in range(len(featuresData)):

        #To be explicit (although techinically unecessary), we will force the same, fixed architecture
        darts_estimator.network_ = fixedArchitecture

        #Determine final DARTS Architecture 
        darts_estimator.set_params(
            max_epochs = 0,
            arch_updates_per_epoch = 0,
            num_graph_nodes = 1,
            param_learning_rate_max = 0.025,
            arch_weight_decay_df = 0.01,
            arch_learning_rate_max = 0.01
        )

        #Split current data
        X_test = featuresData[participant,1:]
        X_test = X_test.reshape(1, -1)
        y_test = featuresData[participant,0]

        X_train = featuresData[:,1:]
        X_train = np.delete(X_train, participant, 0)
        y_train = featuresData[:,0]
        y_train = np.delete(y_train, participant)

        #Fit the new coefficients 
        darts_estimator.fit(X_train, y_train)

        #Evaluate the model
        y_predicted = darts_estimator.predict(X_test)
        accuracy = 1 - np.mean(np.abs(np.round(y_predicted.T)-y_test))
        accuracies.append(accuracy)
        
    #Print and track accuracy
    clear_output(wait=True)
    print(round(np.mean(accuracies)*100,2))
    avgAccuracies.append(round(np.mean(accuracies)*100,2))

    #Keep track of all accuracies computes
    with open('williamsBrainstormChallenge_SearchArchitecture.txt', 'a') as f:
        stringToWrite = str(round(np.mean(accuracies)*100,4)) + '\n'
        f.write(stringToWrite)

    #Only accept architectures with 70% or better predictions
    if np.mean(accuracies)*100 >= 70:
        filename = 'williamsBrainstormChallenge_Architecture.pickle'
        with open(filename, 'wb') as handle:
            pickle.dump(fullArchitecture, handle, protocol=pickle.HIGHEST_PROTOCOL)
        break

In [ ]:
#Step 4. Display Architecture 
print(fullArchitecture.model_repr())
fullArchitecture.visualize_model()

In [ ]:
#Step 5. Extract 3 samples to demonstrate changing coefficients
sampledParticipants = [10, 20, 30]

for participant in sampledParticipants:
        #To be explicit (although techinically unecessary), we will force the same, fixed architecture
        darts_estimator.network_ = fixedArchitecture

        #Determine final DARTS Architecture 
        darts_estimator.set_params(
            max_epochs = 0,
            arch_updates_per_epoch = 0,
            num_graph_nodes = 1,
            param_learning_rate_max = .025,
            arch_weight_decay_df = .01,
            arch_learning_rate_max = .01
        )

        #Split current data
        X_test = featuresData[participant,1:]
        X_test = X_test.reshape(1, -1)
        y_test = featuresData[participant,0]

        X_train = featuresData[:,1:]
        X_train = np.delete(X_train, participant, 0)
        y_train = featuresData[:,0]
        y_train = np.delete(y_train, participant)

        #Fit the new coefficients 
        darts_estimator.fit(X_train, y_train)

        print(darts_estimator.model_repr())
        darts_estimator.visualize_model()